Instalando pacotes

Transformers

In [1]:
! pip install transformers

     |████████████████████████████████| 2.9 MB 5.0 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
     |████████████████████████████████| 895 kB 60.0 MB/s 
     |████████████████████████████████| 636 kB 78.6 MB/s 
     |████████████████████████████████| 3.3 MB 65.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
! pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.0 MB/s 


Dataset

In [3]:
! pip install datasets

     |████████████████████████████████| 270 kB 4.9 MB/s 
     |████████████████████████████████| 243 kB 61.7 MB/s 
     |████████████████████████████████| 125 kB 73.3 MB/s 
     |████████████████████████████████| 1.3 MB 60.7 MB/s 
     |████████████████████████████████| 294 kB 63.2 MB/s 
     |████████████████████████████████| 160 kB 73.7 MB/s 


In [4]:
! pip install simpletransformers

     |████████████████████████████████| 231 kB 5.0 MB/s 
     |████████████████████████████████| 1.7 MB 65.1 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 8.3 MB 25.9 MB/s 
     |████████████████████████████████| 97 kB 9.2 MB/s 
     |████████████████████████████████| 180 kB 65.3 MB/s 
     |████████████████████████████████| 139 kB 72.5 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 111 kB 73.5 MB/s 
     |████████████████████████████████| 76 kB 6.6 MB/s 
     |████████████████████████████████| 4.3 MB 30.3 MB/s 
     |████████████████████████████████| 124 kB 72.3 MB/s 
     |████████████████████████████████| 788 kB 75.8 MB/s 
     |████████████████████████████████| 370 kB 77.1 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=a18750c2e787e0a5dbe71108bd52099f2e7bc325d6f44630a4591fe3921d9ac2
  Stored in directory: /root/.cache/

Googletranslation

In [5]:
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 55 kB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 10.3 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 65 kB 4.8 MB/s 
     |████████████████████████████████| 53 kB 2.5 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=8d65cf5dddd365e822fd7d4e232b1d8656863f096016aad0fe87b39563d1505b
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [6]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 3.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=d2b36d6a2196236c917a856f470cbef1f89fe85fe594a75d854ca49cb556dee1
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [7]:
!pip install datasets

In [8]:
!pip install tensorboardX

     |████████████████████████████████| 124 kB 4.9 MB/s 


Baixando o Dataset

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [1]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

In [2]:
dataset = load_dataset('tapaco', 'pt')

Reusing dataset tapaco (/root/.cache/huggingface/datasets/tapaco/pt/1.0.0/71d200534b520a174927a8f0479c06220a0a6fb5201a84ebfce19006c6354698)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
tapaco = []
# The dataset has only train split.
for data in tqdm(dataset["train"]):
    keys = data.keys()
    tapaco.append([data[key] for key in keys])


100%|██████████| 78430/78430 [00:07<00:00, 10618.42it/s]


In [4]:
tapaco_df = pd.DataFrame(
    data=tapaco,
    columns=[
       "paraphrase_set_id",
       "sentence_id",
        "paraphrase",
        "lists",
        "tags",
        "language",
    ],
)

In [5]:
tapaco_df.head()

,paraphrase_set_id,sentence_id,paraphrase,lists,tags,language
0,1,421918,Eu comi o queijo.,[],[],pt
1,1,5045315,Eu estou comendo queijo.,[],[],pt
2,1,5045316,Estou comendo queijo.,[],[],pt
3,5,620684,Hoje é segunda-feira.,[],[OK],pt
4,5,1227813,É segunda.,[4138],[],pt


In [6]:
def generate_tapaco_paraphrase_dataset(dataset):
    dataset_df = dataset[["paraphrase", "paraphrase_set_id"]]
    non_single_labels = (
        dataset_df["paraphrase_set_id"]
        .value_counts()[dataset_df["paraphrase_set_id"].value_counts() > 1]
        .index.tolist()
    )
    tapaco_df_sorted = dataset_df.loc[
        dataset_df["paraphrase_set_id"].isin(non_single_labels)
    ]
    tapaco_paraphrases_dataset = []

    for paraphrase_set_id in tqdm(tapaco_df_sorted["paraphrase_set_id"].unique()):
        id_wise_paraphrases = tapaco_df_sorted[
            tapaco_df_sorted["paraphrase_set_id"] == paraphrase_set_id
        ]
        len_id_wise_paraphrases = (
            id_wise_paraphrases.shape[0]
            if id_wise_paraphrases.shape[0] % 2 == 0
            else id_wise_paraphrases.shape[0] - 1
        )
        for ix in range(0, len_id_wise_paraphrases, 2):
            current_phrase = id_wise_paraphrases.iloc[ix][0]
            for count_ix in range(ix + 1, ix + 2):
                next_phrase = id_wise_paraphrases.iloc[ix + 1][0]
                tapaco_paraphrases_dataset.append([current_phrase, next_phrase])
    tapaco_paraphrases_dataset_df = pd.DataFrame(
        tapaco_paraphrases_dataset, columns=["Text", "Paraphrase"]
    )
    
    return tapaco_paraphrases_dataset_df

In [7]:
dataset_df = generate_tapaco_paraphrase_dataset(tapaco_df)
dataset_df.head()

100%|██████████| 29949/29949 [02:40<00:00, 186.14it/s]


,Text,Paraphrase
0,Eu comi o queijo.,Eu estou comendo queijo.
1,Hoje é segunda-feira.,É segunda.
2,É segunda-feira.,Hoje é segunda.
3,Onde se encontra a biblioteca?,Onde é a biblioteca?
4,Você fala inglês?,Você não fala inglês?


In [8]:
from simpletransformers.t5 import T5Model
from sklearn.model_selection import train_test_split
import sklearn

In [9]:
dataset_df.columns = ["input_text","target_text"]
dataset_df["prefix"] = "parafrase"

In [10]:
dataset_df.shape

(36451, 3)

In [11]:
dataset_df.head()

,input_text,target_text,prefix
0,Eu comi o queijo.,Eu estou comendo queijo.,parafrase
1,Hoje é segunda-feira.,É segunda.,parafrase
2,É segunda-feira.,Hoje é segunda.,parafrase
3,Onde se encontra a biblioteca?,Onde é a biblioteca?,parafrase
4,Você fala inglês?,Você não fala inglês?,parafrase


In [12]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/402 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
from tqdm import tqdm
tqdm.pandas()

In [14]:
original_embedding = model.encode(dataset_df['input_text'])
dataset_df['original_embedding'] = [original_embedding[i] for i in range(original_embedding.shape[0])]


parafrase_embedding = model.encode(dataset_df['target_text'])
dataset_df['parafrase_embedding'] = [parafrase_embedding[i] for i in range(parafrase_embedding.shape[0])]

dataset_df['similiaridade_parafrase'] = dataset_df.progress_apply(lambda x:util.cos_sim(x['original_embedding'], x['parafrase_embedding']).tolist()[0][0], axis=1)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")
100%|██████████| 36451/36451 [00:03<00:00, 10360.07it/s]


In [15]:
dataset_df.head()

,input_text,target_text,prefix,original_embedding,parafrase_embedding,similiaridade_parafrase
0,Eu comi o queijo.,Eu estou comendo queijo.,parafrase,"[-0.085935086, 0.04360156, -0.003897462, -0.05...","[-0.05510287, -0.027534671, -0.0024288862, -0....",0.882763
1,Hoje é segunda-feira.,É segunda.,parafrase,"[-0.06608112, -0.17827009, -0.011088491, -0.09...","[-0.04838354, -0.1305625, -0.014201537, -0.070...",0.958786
2,É segunda-feira.,Hoje é segunda.,parafrase,"[-0.07405615, -0.15982755, -0.010273912, -0.08...","[-0.060619485, -0.17244618, -0.012098313, -0.0...",0.956231
3,Onde se encontra a biblioteca?,Onde é a biblioteca?,parafrase,"[0.09929005, 0.29744226, -0.006044928, 0.05992...","[0.11998385, 0.3395866, -0.0058360873, 0.04559...",0.991387
4,Você fala inglês?,Você não fala inglês?,parafrase,"[-0.224415, 0.18841593, -0.012384672, -0.03341...","[-0.17771542, 0.25026986, -0.013931634, -0.019...",0.924461


In [16]:
dataset_df['similiaridade_parafrase'].describe()

count    36451.000000
mean         0.920216
std          0.119332
min         -0.136561
25%          0.896255
50%          0.970207
75%          0.994300
max          0.999963
Name: similiaridade_parafrase, dtype: float64

In [17]:
fim = dataset_df[dataset_df['similiaridade_parafrase'] >= 0.8].copy()
fim = fim [["input_text","target_text", "prefix"]].copy()

In [18]:
train_data,test_data = train_test_split(dataset_df,test_size=0.1)

In [19]:
train_data.shape

(32805, 6)

In [20]:
args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 256,
    "num_train_epochs": 4,
    "num_beams": None,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.85,
    "use_multiprocessing": False,
    "save_steps": -1,
    "save_eval_checkpoints": False,
    "evaluate_during_training": False,
    'adam_epsilon': 1e-08,
    'eval_batch_size': 6,
    'fp_16': True,
    'gradient_accumulation_steps': 16,
    'learning_rate': 0.0003,
    'max_grad_norm': 1.0,
    'n_gpu': 1,
    'seed': 42,
    'train_batch_size': 6,
    'warmup_steps': 0,
    'weight_decay': 0.0
}

In [21]:
model = T5Model("t5",'unicamp-dl/ptt5-base-portuguese-vocab', args=args)

Downloading:   0%|          | 0.00/456 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/738k [00:00<?, ?B/s]

In [22]:
model.train_model(train_data, eval_data=test_data, use_cuda=True,acc=sklearn.metrics.accuracy_score)

  0%|          | 0/32805 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3399: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Using Adafactor for T5


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/5468 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/5468 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/5468 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/5468 [00:00<?, ?it/s]

(1364, 1.2167284926208408)

In [23]:
prefix = "parafrase"
pred = model.predict([f"{prefix}: Adoro treinar modelos no python."])
print(pred)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['Eu gosto de treinar modelos no python.']


In [24]:
!zip -r /content/modelo_t5.zip /content/outputs

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  adding: content/outputs/ (stored 0%)
  adding: content/outputs/checkpoint-682-epoch-2/ (stored 0%)
  adding: content/outputs/checkpoint-682-epoch-2/spiece.model (deflated 52%)
  adding: content/outputs/checkpoint-682-epoch-2/optimizer.pt (deflated 15%)
  adding: content/outputs/checkpoint-682-epoch-2/special_tokens_map.json (deflated 83%)
  adding: content/outputs/checkpoint-682-epoch-2/config.json (deflated 45%)
  adding: content/outputs/checkpoint-682-epoch-2/model_args.json (deflated 63%)
  adding: content/outputs/checkpoint-682-epoch-2/pytorch_model.bin (deflated 7%)
  adding: content/outputs/checkpoint-682-epoch-2/scheduler.pt (deflated 51%)
  adding: content/o

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [25]:
from google.colab import files
files.download("/content/modelo_t5.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install huggingface_hub

!huggingface-cli login

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_

In [ ]:
!huggingface-cli login

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: lucasfaopellicer
Password: 
Login successful
Your token has been

In [ ]:
model.push_to_hub("ptt5_finetuning_paraphrasing")

AttributeError: ignored

In [ ]:
ppdb = pd.read_csv('/content/drive/My Drive/Colab Notebooks/parafraseadores/ppdb-1.0-xxl-phrasal',sep='\t')
ppdb.shape

(8488778, 1)

In [ ]:
ppdb[ppdb.columns[0]].iloc[2].split('|||')

['[NP/NP] ',
 ' de sete propostas ',
 ' sobre sete propostas ',
 ' Abstract=0 Adjacent=0 Alignment=0-0:1-1:2-2 CharCountDiff=3 CharLogCR=0.16252 ContainsX=0 GlueRule=0 Identity=0 Lex(e|f)=7.44420 Lex(f|e)=9.75340 Lexical=1 LogCount=0 Monotonic=1 PhrasePenalty=1 RarityPenalty=0.01832 SourceTerminalsButNoTarget=0 SourceWords=3 TargetTerminalsButNoSource=0 TargetWords=3 UnalignedSource=0 UnalignedTarget=0 WordCountDiff=0 WordLenDiff=1.00000 WordLogCR=0 p(LHS|e)=0 p(LHS|f)=0 p(e|LHS)=14.59258 p(e|f)=0.69315 p(e|f,LHS)=0.69315 p(f|LHS)=14.59258 p(f|e)=0.69315 p(f|e,LHS)=0.69315']

In [ ]:
ppdb.columns

Index(['[ADJP/S] ||| satisfeita por termos ||| satisfeito por termos ||| Abstract=0 Adjacent=0 Alignment=0-0:0-1:1-0:1-1:1-1:2-2:2-2 CharCountDiff=0 CharLogCR=0 ContainsX=0 GlueRule=0 Identity=0 Lex(e|f)=21.74269 Lex(f|e)=21.08456 Lexical=1 LogCount=0 Monotonic=1 PhrasePenalty=1 RarityPenalty=0.01832 SourceTerminalsButNoTarget=0 SourceWords=3 TargetTerminalsButNoSource=0 TargetWords=3 UnalignedSource=0 UnalignedTarget=0 WordCountDiff=0 WordLenDiff=0 WordLogCR=0 p(LHS|e)=0 p(LHS|f)=0 p(e|LHS)=10.65267 p(e|f)=0.69315 p(e|f,LHS)=0.69315 p(f|LHS)=10.65267 p(f|e)=0.69315 p(f|e,LHS)=0.69315'], dtype='object')

In [ ]:
import pandas as pd

dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/semeval_ironic/SemEval2018-T3-train-taskA_emoji.txt',sep = '\t')
dataset.head()

,Tweet index,Label,Tweet text
0,1,1,Sweet United Nations video. Just in time for C...
1,2,1,@mrdahl87 We are rumored to have talked to Erv...
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...
3,4,0,3 episodes left I'm dying over here
4,5,1,I can't breathe! was chosen as the most notabl...


In [ ]:
import nltk.data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader

# Define your sentence transformer model using CLS pooling
model_name = 'bert-base-uncased'
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# Define a list with sentences (1k - 100k sentences)
train_sentences = dataset['Tweet text'].values
# Create the special denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)

# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
When tie_encoder_decoder=True, the decoder_name_or_path will be invalid.
Some weights of the model checkpoint at bert-base-uncased were not used when initi

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/478 [00:00<?, ?it/s]

In [ ]:
model.encode(['The book is on the table'])

array([[-3.52198750e-01,  6.17730692e-02, -6.19642362e-02,
        -1.14088401e-01, -4.91785642e-04, -4.53151882e-01,
        -3.31832379e-01,  3.58754516e-01, -5.40511496e-02,
        -3.81678402e-01,  2.56333411e-01,  1.45293057e-01,
         1.70565516e-01,  1.17001012e-01,  1.41167805e-01,
        -2.77859390e-01,  1.46155655e-01,  7.90150344e-01,
        -7.43580759e-02, -2.67987221e-01,  1.51682079e-01,
        -6.56632856e-02, -1.07062422e-02, -1.77936986e-01,
         1.06728286e-01, -2.46589646e-01, -3.07748020e-01,
        -5.59899397e-02, -8.97997711e-03,  1.75802603e-01,
        -1.80283096e-02, -2.61524677e-01,  4.07436281e-01,
        -1.52887627e-01,  5.10655463e-01, -1.61542639e-01,
         1.21602505e-01,  1.58442557e-01,  1.55604705e-01,
         1.49828466e-02, -1.86536405e-02, -3.46690297e-01,
         7.39718899e-02,  1.05669878e-01, -1.62161022e-01,
        -3.77952486e-01, -1.61057532e+00,  1.73663467e-01,
         1.26996085e-01, -2.76747532e-02, -3.77514889e-0

In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
model.modules

<bound method Module.modules of SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)>

In [ ]:
model.

<bound method Module.modules of SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)>

In [ ]:
from transformers import T5Tokenizer

# PyTorch (bare model, baremodel + language modeling head)
from transformers import T5Model, T5ForConditionalGeneration

# Tensorflow (bare model, baremodel + language modeling head)
from transformers import TFT5Model, TFT5ForConditionalGeneration


In [ ]:
model_name = 'unicamp-dl/ptt5-base-portuguese-vocab'

tokenizer = T5Tokenizer.from_pretrained(model_name)

# PyTorch
model_pt = T5ForConditionalGeneration.from_pretrained(model_name)



Downloading:   0%|          | 0.00/756k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at unicamp-dl/ptt5-base-portuguese-vocab.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
type(tokenizer)

transformers.models.t5.tokenization_t5.T5Tokenizer

In [ ]:
context = r"""
A pandemia de COVID-19, também conhecida como pandemia de coronavírus, é uma pandemia em curso de COVID-19, 
uma doença respiratória aguda causada pelo coronavírus da síndrome respiratória aguda grave 2 (SARS-CoV-2). 
A doença foi identificada pela primeira vez em Wuhan, na província de Hubei, República Popular da China, 
em 1 de dezembro de 2019, mas o primeiro caso foi reportado em 31 de dezembro do mesmo ano. 
Acredita-se que o vírus tenha uma origem zoonótica, porque os primeiros casos confirmados 
tinham principalmente ligações ao Mercado Atacadista de Frutos do Mar de Huanan, que também vendia animais vivos. 
Em 11 de março de 2020, a Organização Mundial da Saúde declarou o surto uma pandemia. Até 8 de fevereiro de 2021, 
pelo menos 105 743 102 casos da doença foram confirmados em pelo menos 191 países e territórios, 
com cerca de 2 308 943 mortes e 58 851 440 pessoas curadas.
"""

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
text =  "summarize: " + context

max_len = 256

encoding = tokenizer.encode_plus(text, return_tensors = "pt")
input_ids = encoding["input_ids"].to(device)
attention_masks = encoding["attention_mask"].to(device)

beam_outputs = model_pt.generate(
    input_ids = input_ids,
    attention_mask = attention_masks,
    max_length = 64,
    num_beams = 3,
    early_stopping = True,
)

result = tokenizer.decode(beam_outputs[0])
print(result)

RuntimeError: ignored

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('adalbertojunior/PTT5-SMALL-SUM')

t5 = T5ForConditionalGeneration.from_pretrained('adalbertojunior/PTT5-SMALL-SUM')

text="Esse é um exemplo de sumarização."

input_ids = tokenizer.encode(context, return_tensors="pt", add_special_tokens=True)

generated_ids = t5.generate(
        input_ids=input_ids,
        num_beams=1,
        max_length=100,
        #repetition_penalty=2.5
    ).squeeze()
    
predicted_span = tokenizer.decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)


In [ ]:
predicted_span

'A pandemia de COVID-19, também conhecida como pandemia de coronavírus, é uma pandemia em curso de COVID-19, uma doença respiratória aguda causada pelo coronavírus da síndrome respiratória aguda grave 2 (SARS-CoV-2). A doença foi identificada pela primeira vez em Wuhan, na província de Hubei, República Popular da China, em 1 de dezembro de 2019, mas o primeiro caso foi reportado em 31 de dezembro do mesmo ano.'

In [ ]:
result = tokenizer.decode(beam_outputs[0])
print(result)